# Scaling up matrix multiplication

In [1]:
import dask.array as da
from dask.distributed import Client
import time
from ..scale.scale_cluster import *
#from scale_cluster import *

### Settings Init Cluster

In [2]:
max_workers = 16  # It depends settings cluster
max_cores_by_workers = 24

# Settings Init Matrix
n_rows_x = 1000
n_col_x = 2000
n_rows_y = 2000
n_col_y = 1000

# Init vars matrix
x_1 = da.ones((n_rows_x,n_col_x), dtype=int)
y_1 = da.ones((n_rows_y,n_col_y), dtype=int)
sequential_time = 4
speed_ups = []

### Get time sequential

In [5]:
init_cluster('170.210.103.20', str(1), str(1))
time.sleep(5)
client = Client('170.210.103.20:8786')
print(client)
# Execute task
t = time.time()
mult = da.matmul(x_1,y_1)
m = client.compute(mult)
res = m.result()

sequential_time = time.time()-t
print('Result:')
print(res)
print('With nodes:1 '+'cores_by_workers: '+str(1)+ '\tTime: '+ str(sequential_time))    
print('----------------------------------------------------------')
kill_cluster('170.210.103.20')
time.sleep(10)

<Client: scheduler='tcp://170.210.103.20:8786' processes=1 cores=1>
Result:
[[2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 ...
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]]
With nodes:1 cores_by_workers: 1	Time: 17.235417127609253
----------------------------------------------------------


### Get Speed ups

In [3]:
for j in range(1,max_workers+1):    
    init_cluster('170.210.103.20', str(j), str(24))
    time.sleep(5)
    client = Client('170.210.103.20:8786')
    print(client)
    # Execute task
    t = time.time()
    mult = da.matmul(x_1,y_1)
    m = client.compute(mult)
    res = m.result()
    t1 = time.time()-t
    speed_ups.append(t1/sequential_time)
    print(res)
    print('Time: '+ str(t1))    
    print('----------------------------------------------------------')
    kill_cluster('170.210.103.20')
    time.sleep(15)

<Client: scheduler='tcp://170.210.103.20:8786' processes=1 cores=24>
[[2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 ...
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]]
Time: 17.180882930755615
----------------------------------------------------------
<Client: scheduler='tcp://170.210.103.20:8786' processes=2 cores=48>
[[2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 ...
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]]
Time: 17.582024812698364
----------------------------------------------------------
<Client: scheduler='tcp://170.210.103.20:8786' processes=3 cores=72>
[[2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 ...
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000

KeyboardInterrupt: 

In [5]:
l = [1, 2, 4, 8, 16, 24]
for j in l:    
    init_cluster('170.210.103.20', str(1), str(j))
    time.sleep(5)
    client = Client('170.210.103.20:8786')
    print(client)
    # Execute task
    t = time.time()
    mult = da.matmul(x_1,y_1)
    m = client.compute(mult)
    res = m.result()
    t1 = time.time()-t
    speed_ups.append(t1/sequential_time)
    print(res)
    print('Time: '+ str(t1))    
    print('----------------------------------------------------------')
    kill_cluster('170.210.103.20')
    time.sleep(15)

<Client: scheduler='tcp://170.210.103.20:8786' processes=1 cores=1>
[[2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 ...
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]]
Time: 17.240084886550903
----------------------------------------------------------
<Client: scheduler='tcp://170.210.103.20:8786' processes=1 cores=2>
[[2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 ...
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]]
Time: 17.980249404907227
----------------------------------------------------------
<Client: scheduler='tcp://170.210.103.20:8786' processes=1 cores=4>
[[2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ... 2000 2000 2000]
 ...
 [2000 2000 2000 ... 2000 2000 2000]
 [2000 2000 2000 ..

### Plot speed ups

In [7]:
import matplotlib.pyplot as plt
cores = 24
x = []
for s in speed_ups:
    x.append(cores)
    cores += 24
plt.plot(x, speed_ups)
plt.ylabel('Speed-Up')
plt.xlabel('Cores')
plt.show()

<Figure size 640x480 with 1 Axes>

In [4]:
kill_cluster('170.210.103.20')